In [1]:
import os
import time
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import *

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

In [2]:
path = os.getcwd() + "/dataset/"
path

'/root/dataset/'

In [3]:
TRAINING_DIR = path+"/train"
VALIDATION_DIR = path + "/val"
TEST_DIR = path + "/test"

print(TRAINING_DIR)
print(VALIDATION_DIR)
print(TEST_DIR)

/root/dataset//train
/root/dataset//val
/root/dataset//test


In [4]:
def data_loader(path):
    X = []
    Y = []
    for folder in os.listdir(path):
        for file in os.listdir(os.path.join(path, folder)):
            feature = np.load(os.path.join(path, folder, file))
            label = folder
            X.append(feature)
            Y.append(label)
    return X, Y

In [5]:
X_train, Y_train = data_loader(TRAINING_DIR)
X_val, Y_val = data_loader(VALIDATION_DIR)
X_test, Y_test = data_loader(TEST_DIR)

In [6]:
print("There are {} training samples and {} training labels".format(len(X_train), len(Y_train)))
print("There are {} validation samples and {} validation labels".format(len(X_val), len(Y_val)))
print("There are {} testing samples and {} testing labels".format(len(X_test), len(Y_test)))

There are 14000 training samples and 14000 training labels
There are 2000 validation samples and 2000 validation labels
There are 4000 testing samples and 4000 testing labels


In [7]:
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_val = np.array(Y_val)
Y_test = np.array(Y_test)

print("Shape of X_train: {}".format(X_train.shape))
print("Shape of X_val: {}".format(X_val.shape))
print("Shape of X_test: {}".format(X_test.shape))

Shape of X_train: (14000, 64575)
Shape of X_val: (2000, 64575)
Shape of X_test: (4000, 64575)


In [8]:
# shuffle the dataset
shuffle_index = np.random.permutation(len(X_train))
X_train, Y_train = X_train[shuffle_index], Y_train[shuffle_index]
shuffle_index = np.random.permutation(len(X_val))
X_val, Y_val = X_val[shuffle_index], Y_val[shuffle_index]
shuffle_index = np.random.permutation(len(X_test))
X_test, Y_test = X_test[shuffle_index], Y_test[shuffle_index]

In [9]:
label_encoder = LabelEncoder()
Y_train = label_encoder.fit_transform(Y_train)
Y_val = label_encoder.fit_transform(Y_val)
Y_test = label_encoder.fit_transform(Y_test)

In [11]:
def model_training(model, X_train, Y_train, X_val, Y_val, X_test, Y_test):
    start_time = time.time()
    model.fit(X_train, Y_train)
    time_taken = time.time() - start_time
    Y_val_pred = model.predict(X_val)
    Y_test_pred = model.predict(X_test)
    val_acc = accuracy_score(Y_val, Y_val_pred)
    test_acc = accuracy_score(Y_test, Y_test_pred)
    recall_val = recall_score(Y_val, Y_val_pred, average='macro')
    recall_test = recall_score(Y_test, Y_test_pred, average='macro')
    precision_val = precision_score(Y_val, Y_val_pred, average='macro')
    precision_test = precision_score(Y_test, Y_test_pred, average='macro')
    f1_val = f1_score(Y_val, Y_val_pred, average='macro')
    f1_test = f1_score(Y_test, Y_test_pred, average='macro')
    conf_matrix_val = confusion_matrix(Y_val, Y_val_pred)
    conf_matrix_test = confusion_matrix(Y_test, Y_test_pred)
    print('We have used the following model: {}'.format(model))
    print('Test Accuracy: {}'.format(test_acc))
    print('Time taken: {}'.format(time_taken))
    return val_acc, test_acc, recall_val, recall_test, precision_val, precision_test, f1_val, f1_test, conf_matrix_val, conf_matrix_test, time_taken

In [12]:
resul = []
model_names = ['Random Forest', 'KNN', 'MLP', 'Logistic Regression', 'Naive Bayes', 'Decision Tree', 'AdaBoost', 'Gradient Boosting', 'Extra Trees', 'Bagging', 'XGBoost']
models = [RandomForestClassifier(), 
          KNeighborsClassifier(), MLPClassifier(), LogisticRegression(), GaussianNB(), DecisionTreeClassifier(), AdaBoostClassifier(), GradientBoostingClassifier(), ExtraTreesClassifier(), BaggingClassifier(), XGBClassifier()]

In [ ]:
for model, model_name in zip(models, model_names):
    val_acc, test_acc, recall_val, recall_test, precision_val, precision_test, f1_val, f1_test, conf_matrix_val, conf_matrix_test, time_taken = model_training(model, X_train, Y_train, X_val, Y_val, X_test, Y_test)
    resul.append({'Model': model_name, 'Validation Accuracy': val_acc, 'Test Accuracy': test_acc, 'Validation Recall': recall_val, 'Test Recall': recall_test, 'Validation Precision': precision_val, 'Test Precision': precision_test, 'Validation F1': f1_val, 'Test F1': f1_test, 'Validation Confusion Matrix': conf_matrix_val, 'Test Confusion Matrix': conf_matrix_test, 'Time Taken': time_taken})

We have used the following model: RandomForestClassifier()
Test Accuracy: 0.8395
Time taken: 444.2838685512543
We have used the following model: KNeighborsClassifier()
Test Accuracy: 0.85125
Time taken: 0.2681307792663574
We have used the following model: MLPClassifier()
Test Accuracy: 0.90775
Time taken: 3225.921051502228
We have used the following model: LogisticRegression()
Test Accuracy: 0.7895
Time taken: 69.44343328475952
We have used the following model: GaussianNB()
Test Accuracy: 0.506
Time taken: 3.9439022541046143


In [ ]:
result_df = pd.DataFrame(resul)
result_df.to_csv('result_1D_STFT.csv', index=False)
result_df.head(12)

In [ ]:
## Without SVM